In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Compose, Resize
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
size = 128
batch_size = 128
transformation = Compose([Resize((size, size)), ToTensor()])

In [5]:
root = 'real_and_fake_face'
dataset = ImageFolder(root, transformation)
data_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = False)

In [6]:
class DeepSearchConv(torch.nn.Module):
    def __init__(self, inputs, outputs, kernel = 3):
        super(DeepSearchConv, self).__init__()
        
        self.conv = nn.Conv2d(inputs, outputs, kernel_size = kernel)
        self.dropout = nn.Dropout2d(0.25)
        self.batch_norm = nn.BatchNorm2d(outputs)
        
    def forward(self, x):
        x = self.conv(x)
        x = F.leaky_relu(x)
        x = self.batch_norm(x)
        x = self.dropout(x)
        return x

In [7]:
class DeepSearchDownsize(torch.nn.Module):
    def __init__(self, inputs, outputs):
        super(DeepSearchDownsize, self).__init__()
        
        self.maxpool = nn.AvgPool2d(2)
        self.linear = nn.Linear(inputs, outputs)
        self.inputs = inputs
        
    def forward(self, x):
        x = self.maxpool(x)
        x = x.view(-1, self.inputs)
        x = self.linear(x)
        return x

In [8]:
class DeepSearchLinear(torch.nn.Module):
    def __init__(self, inputs, outputs):
        super(DeepSearchLinear, self).__init__()
        
        self.batch_norm = nn.BatchNorm1d(inputs)
        self.dropout = nn.Dropout(0.25)
        self.linear = nn.Linear(inputs, outputs)
        
    def forward(self, x):
        x = F.leaky_relu(x)
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [9]:
class DeepSearch(torch.nn.Module):
    def __init__(self, hidden, flattened, linear, labels):
        super(DeepSearch, self).__init__()
        
        self.conv = [ DeepSearchConv( hidden[i], hidden[i+1] ) for i in range(len(hidden)-1) ]
        self.downsize = DeepSearchDownsize(flattened, linear)
        self.output = DeepSearchLinear(linear, labels)
        
        self.combined = nn.Sequential(*self.conv,
                                     self.downsize,
                                     self.output)
    
    def forward(self, x):
        x = self.combined(x)
        return x

In [10]:
from math import floor
hidden = [3, 32, 32]
channels = hidden[-1]
changed_img = floor((size-2*(len(hidden)-1))/2)
linear = 128
labels = 2
flattened = channels*changed_img*changed_img

In [13]:
model = DeepSearch(hidden=hidden, flattened=flattened, linear=linear, labels=labels).to(device)

In [26]:
checkpoint = torch.load('./pytorch_models/0.759acc_Adam0.05_amsgrad')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
test_losses = []
test_accuracies = []

def test(model, data_loader):
    # Use cross-entropy loss function
    criterion = nn.CrossEntropyLoss()
    model.eval()
    # Initialize epoch loss and accuracy
    epoch_loss = 0.0
    correct = 0
    total = 0
    # Iterate over test data
    for inputs, labels in data_loader:
        # Get from dataloader and send to device
        inputs = inputs.to(device)
        labels = labels.to(device)
        # Compute model output and loss
        # (No grad computation here, as it is the test data)
        with torch.no_grad():
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
        # Accumulate loss and correct predictions for epoch
        epoch_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # Calculate epoch loss and accuracy
    epoch_loss /= len(data_loader)
    epoch_acc = correct/total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)
    print(f'Test loss: {epoch_loss:.4f}, Test accuracy: {epoch_acc:.4f}')
    return test_losses, test_accuracies

In [27]:
test_losses, test_accuracies = test(model, data_loader)

Test loss: 0.6249, Test accuracy: 0.6570
